In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kyc-ds/sdn.csv
/kaggle/input/kyc-comp/sdn.csv
/kaggle/input/kyc-comp/id_2.pdf
/kaggle/input/kyc-comp/id_1.png
/kaggle/input/sample-docs/id_2.pdf
/kaggle/input/sample-docs/id_1.png
/kaggle/input/final-kyc/id_2.pdf
/kaggle/input/final-kyc/id_1.png
/kaggle/input/final-kyc/sdn_clean.csv


In [22]:
#TO CLEAN THE ORGININAL DATASET

# raw_path = "/kaggle/input/kyc-ds/sdn.csv" 


# df = pd.read_csv(raw_path, dtype=str, low_memory=False, on_bad_lines="skip")
# print("Original shape:", df.shape)
# print("Columns:", list(df.columns))


# possible_name_cols = [c for c in df.columns if "NAME" in c.upper()]
# print("Possible name cols:", possible_name_cols)


# df_clean = pd.DataFrame()
# df_clean["name"] = df.iloc[:, 1]  # column B (name)
# df_clean["type"] = df.iloc[:, 2]  # column C
# df_clean["program"] = df.iloc[:, 3]  # column D


# try:
#     df_clean["details"] = df.iloc[:, 11]  # column L (DOB info)
# except Exception as e:
#     print("Couldn't extract DOB/POB column:", e)
#     df_clean["details"] = ""


# df_clean = df_clean.dropna(subset=["name"]).reset_index(drop=True)


# df_clean = df_clean.head(5000)


# out_path = "/kaggle/working/sdn_clean.csv"
# df_clean.to_csv(out_path, index=False)

# df_clean.head()


In [23]:
# df_clean.to_csv('Sdn_clean.csv', index=False)

In [24]:
!apt-get update -qq && apt-get install -y -qq tesseract-ocr poppler-utils

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [25]:
!pip install -q --upgrade "pyarrow>=21.0.0" "pydantic<2.12,>=2.0"

In [26]:
!pip install -q pytesseract pdfplumber pillow rapidfuzz transformers sentencepiece

In [27]:
!apt-get update -qq && apt-get install -y -qq tesseract-ocr poppler-utils
!pip install -q pytesseract pdfplumber pillow rapidfuzz transformers sentencepiece

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [28]:
import sys, subprocess
import pytesseract, pdfplumber, PIL, rapidfuzz, transformers

print("Python:", sys.version.splitlines()[0])
print("pytesseract:", pytesseract.__version__)
print("pdfplumber:", pdfplumber.__version__)
print("rapidfuzz:", rapidfuzz.__version__)
print("transformers:", transformers.__version__)

print("\nTesseract binary version:")
subprocess.run(["tesseract", "--version"])


Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
pytesseract: 0.3.13
pdfplumber: 0.11.7
rapidfuzz: 3.14.1
transformers: 4.53.3

Tesseract binary version:
tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


CompletedProcess(args=['tesseract', '--version'], returncode=0)

In [29]:
import os

DATA_DIR = "/kaggle/input/kyc-comp"  

for root, dirs, files in os.walk(DATA_DIR):
    level = root.replace(DATA_DIR, "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}  - {f}")


kyc-comp/
  - sdn.csv
  - id_2.pdf
  - id_1.png


In [30]:
from PIL import Image
import pytesseract, pdfplumber
import pandas as pd
from rapidfuzz import fuzz, process
import os

In [31]:
DATA_DIR = "/kaggle/input/final-kyc" 
img_path = os.path.join(DATA_DIR, "id_1.png")  
pdf_path = os.path.join(DATA_DIR, "id_2.pdf")
sdn_path = os.path.join(DATA_DIR, "sdn_clean.csv")

In [32]:
if os.path.exists(img_path):
    print("OCR image test —", img_path)
    text = pytesseract.image_to_string(Image.open(img_path))
    print("---- OCR output (first 300 chars) ----")
    print(text[:300])
else:
    print("Image not found at", img_path)

OCR image test — /kaggle/input/final-kyc/id_1.png
---- OCR output (first 300 chars) ----
NAME: John A. Doe



In [33]:
if os.path.exists(pdf_path):
    import os, subprocess
from PIL import Image
import pytesseract

pdf_path = "/kaggle/input/final-kyc/id_2.pdf"
tmp_dir = "/kaggle/working/pdf_images"
os.makedirs(tmp_dir, exist_ok=True)


subprocess.run(
    ["pdftoppm", "-png", pdf_path, os.path.join(tmp_dir, "page")],
    check=True
)


texts = []
for fname in sorted(os.listdir(tmp_dir)):
    if fname.endswith(".png"):
        path = os.path.join(tmp_dir, fname)
        text = pytesseract.image_to_string(Image.open(path))
        texts.append(text)
        print(f"OCR from {fname}:\n{text[:200]}\n{'-'*50}")


pdf_text = "\n".join(texts)
print("Final combined OCR text :\n", pdf_text[:500])


OCR from page-1.png:
 

NAME: Jane Q. Public

--------------------------------------------------
Final combined OCR text :
  

NAME: Jane Q. Public



In [34]:
if os.path.exists(sdn_path):
    sdn = pd.read_csv(sdn_path, dtype=str).fillna("")
    names = sdn['name'].tolist()
    candidate = "John A. Doe"
    print("\nFuzzy match test for:", candidate)
    match = process.extractOne(candidate, names, scorer=fuzz.WRatio)
    print("Best match:", match)
else:
    print("sdn.csv not found at", sdn_path)


Fuzzy match test for: John A. Doe
Best match: ('OGUNGBUYI, Oluwole A.', 85.5, 119)


In [35]:
from PIL import Image
import pytesseract, pdfplumber
import pandas as pd
from rapidfuzz import fuzz, process
import os


DATA_DIR = "/kaggle/input/final-kyc"  
sdn_path = os.path.join(DATA_DIR, "sdn_clean.csv")


if os.path.exists(sdn_path):
    sdn = pd.read_csv(sdn_path, dtype=str).fillna("")
    print(f"Loaded SDN file with {len(sdn)} rows and columns: {list(sdn.columns)}")
else:
    raise FileNotFoundError(f"Could not find {sdn_path}")


candidate = "John A. Doe" 
names = sdn["name"].tolist()

match = process.extractOne(candidate, names, scorer=fuzz.WRatio)

print("\n🔍 Fuzzy match test result:")
print(f"Candidate name: {candidate}")
print(f"Best match: {match[0]}")
print(f"Match score: {match[1]:.2f}")
print(f"Row index: {match[2]}")
print("\nMatching row details:")
print(sdn.iloc[match[2]])


Loaded SDN file with 5000 rows and columns: ['name', 'type', 'program', 'details']

🔍 Fuzzy match test result:
Candidate name: John A. Doe
Best match: OGUNGBUYI, Oluwole A.
Match score: 85.50
Row index: 119

Matching row details:
name               OGUNGBUYI, Oluwole A.
type                          individual
program                            SDNTK
details    DOB 04 Mar 1953; POB Nigeria.
Name: 119, dtype: object


In [36]:
import pandas as pd, time, os
from PIL import Image
import pytesseract
from rapidfuzz import process, fuzz


SDN_PATH = "/kaggle/input/final-kyc/sdn_clean.csv"
INCOMING_PATH = "/kaggle/working/incoming_names.csv" 


data = [
    {"record_id": "r1", "name": "John A. Doe", "doc_path": "/kaggle/input/kyc-seed-data-yourname/sample_docs/id_1.png"},
    {"record_id": "r2", "name": "Jane Q. Public", "doc_path": "/kaggle/input/kyc-seed-data-yourname/sample_docs/id_2.pdf"},
    {"record_id": "r3", "name": "Jonathan Doe", "doc_path": ""},
    {"record_id": "r4", "name": "Ali Ahmed", "doc_path": ""},
]
pd.DataFrame(data).to_csv(INCOMING_PATH, index=False)
print(f"Demo input stream saved to {INCOMING_PATH}")


sdn = pd.read_csv(SDN_PATH, dtype=str).fillna("")
print(f"Loaded SDN dataset with {len(sdn)} rows")


incoming = pd.read_csv(INCOMING_PATH)
print(f"Loaded {len(incoming)} incoming records\n")


def extract_name_from_doc(path):
    if not path or not os.path.exists(path):
        return ""
    text = ""
    if path.lower().endswith(".pdf"):
        
        import subprocess
        tmp_dir = "/kaggle/working/tmp_pdf"
        os.makedirs(tmp_dir, exist_ok=True)
        subprocess.run(["pdftoppm", "-r", "200", "-png", path, os.path.join(tmp_dir, "page")], check=True)
        for f in sorted(os.listdir(tmp_dir)):
            if f.endswith(".png"):
                text += pytesseract.image_to_string(Image.open(os.path.join(tmp_dir, f))) + "\n"
    else:
        text = pytesseract.image_to_string(Image.open(path))
    return text


print(" Starting simulated stream...\n")
for _, row in incoming.iterrows():
    print(f" New record arrived: {row['record_id']} — {row['name']}")
    candidate_name = row["name"]
    
   
    if not candidate_name and row["doc_path"]:
        candidate_name = extract_name_from_doc(row["doc_path"])
    
 
    match = process.extractOne(candidate_name, sdn["name"], scorer=fuzz.WRatio)
    
    if match and match[1] >= 85:
        matched_row = sdn.iloc[match[2]]
        print(f" Potential match found!  Score: {match[1]:.1f}")
        print(f"Matched SDN entry: {matched_row['name']} ({matched_row['program']})")
        print(f"Details: {matched_row['details']}\n")
    else:
        print(" No relevant match found.\n")
    
    time.sleep(2)  # simulate delay between events

print(" Stream simulation complete!")


Demo input stream saved to /kaggle/working/incoming_names.csv
Loaded SDN dataset with 5000 rows
Loaded 4 incoming records

 Starting simulated stream...

 New record arrived: r1 — John A. Doe
 Potential match found!  Score: 85.5
Matched SDN entry: OGUNGBUYI, Oluwole A. (SDNTK)
Details: DOB 04 Mar 1953; POB Nigeria.

 New record arrived: r2 — Jane Q. Public
 No relevant match found.

 New record arrived: r3 — Jonathan Doe
 Potential match found!  Score: 85.5
Matched SDN entry: MUNOZ MEJIA, Jonathan (SDNTK)
Details: DOB 07 Nov 1985; POB Manizales, Caldas, Colombia; nationality Colombia; Cedula No. 75107204 (Colombia); C.U.R.P. MUMJ851107HNEXJN01 (Mexico); Linked To: AVICAL S.A.; Linked To: INVERSIONES LA PLATA M & M S. EN C.A.; Linked To: ROMIK S.A.; Linked To: MUNSA INTERNATIONAL INVESMENTS S.A.

 New record arrived: r4 — Ali Ahmed
 Potential match found!  Score: 90.0
Matched SDN entry: AL-BADANI, Shawki Ali Ahmed (SDGT)
Details: DOB 01 Jan 1981; alt. DOB 01 Jan 1982; POB Sana, Yemen; n

In [37]:
import pandas as pd


data = [
    {
        "record_id": "r1",
        "name": "John A. Doe",
        "doc_path": "/kaggle/input/kyc-seed-data-yourname/sample_docs/id_1.png"
    },
    {
        "record_id": "r2",
        "name": "Jane Q. Public",
        "doc_path": "/kaggle/input/kyc-seed-data-yourname/sample_docs/id_2.pdf"
    },
    {
        "record_id": "r3",
        "name": "Jonathan Doe",
        "doc_path": ""
    },
    {
        "record_id": "r4",
        "name": "Ali Ahmed",
        "doc_path": ""
    },
]

incoming_df = pd.DataFrame(data)
incoming_df.to_csv("/kaggle/working/incoming_names.csv", index=False)
print("Created /kaggle/working/incoming_names.csv")
print(incoming_df)


Created /kaggle/working/incoming_names.csv
  record_id            name                                           doc_path
0        r1     John A. Doe  /kaggle/input/kyc-seed-data-yourname/sample_do...
1        r2  Jane Q. Public  /kaggle/input/kyc-seed-data-yourname/sample_do...
2        r3    Jonathan Doe                                                   
3        r4       Ali Ahmed                                                   


In [38]:
import pandas as pd, time, datetime, os
from rapidfuzz import process, fuzz
from PIL import Image
import pytesseract, subprocess


SDN_PATH = "/kaggle/input/final-kyc/sdn_clean.csv"
INCOMING_PATH = "/kaggle/working/incoming_names.csv"
STATE_PATH = "/kaggle/working/kyc_state.csv"


def extract_name_from_doc(path):
    if not path or not os.path.exists(path):
        return ""
    text = ""
    if path.lower().endswith(".pdf"):
        tmp_dir = "/kaggle/working/tmp_pdf"
        os.makedirs(tmp_dir, exist_ok=True)
        subprocess.run(["pdftoppm", "-r", "200", "-png", path, os.path.join(tmp_dir, "page")], check=True)
        for f in sorted(os.listdir(tmp_dir)):
            if f.endswith(".png"):
                text += pytesseract.image_to_string(Image.open(os.path.join(tmp_dir, f))) + "\n"
    else:
        text = pytesseract.image_to_string(Image.open(path))
    return text.strip()


sdn = pd.read_csv(SDN_PATH, dtype=str).fillna("")
incoming = pd.read_csv(INCOMING_PATH, dtype=str).fillna("")


if os.path.exists(STATE_PATH):
    state = pd.read_csv(STATE_PATH, dtype=str)
else:
    state = pd.DataFrame(columns=[
        "record_id", "input_name", "matched_name", "score", 
        "program", "details", "status", "timestamp"
    ])

print(f"Loaded {len(state)} previous results\n")


print("Starting incremental streaming simulation...\n")

for _, row in incoming.iterrows():
    record_id = row["record_id"]
    name_in = row["name"]
    
   
    if record_id in state["record_id"].values:
        print(f"Record {record_id} already processed, skipping.")
        continue
    
    print(f"Processing record {record_id}: {name_in}")
    candidate = name_in or extract_name_from_doc(row["doc_path"])
    
    match = process.extractOne(candidate, sdn["name"], scorer=fuzz.WRatio)
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if match and match[1] >= 85:
        matched_row = sdn.iloc[match[2]]
        result = {
            "record_id": record_id,
            "input_name": candidate,
            "matched_name": matched_row["name"],
            "score": f"{match[1]:.1f}",
            "program": matched_row["program"],
            "details": matched_row["details"],
            "status": "FLAGGED",
            "timestamp": timestamp
        }
        print(f"Match found ({match[1]:.1f}): {matched_row['name']} — {matched_row['program']}")
    else:
        result = {
            "record_id": record_id,
            "input_name": candidate,
            "matched_name": "",
            "score": "",
            "program": "",
            "details": "",
            "status": "CLEAR",
            "timestamp": timestamp
        }
        print("No relevant match found.")
    
    # append to state
    state = pd.concat([state, pd.DataFrame([result])], ignore_index=True)
    state.to_csv(STATE_PATH, index=False)
    
    print(f"State updated. Total processed: {len(state)}\n")
    time.sleep(2)  # simulate delay

print("Streaming loop complete!")
print("\nFinal state summary:\n", state.tail(5))


Loaded 4 previous results

Starting incremental streaming simulation...

Record r1 already processed, skipping.
Record r2 already processed, skipping.
Record r3 already processed, skipping.
Record r4 already processed, skipping.
Streaming loop complete!

Final state summary:
   record_id      input_name                 matched_name score program  \
0        r1     John A. Doe        OGUNGBUYI, Oluwole A.  85.5   SDNTK   
1        r2  Jane Q. Public                          NaN   NaN     NaN   
2        r3    Jonathan Doe        MUNOZ MEJIA, Jonathan  85.5   SDNTK   
3        r4       Ali Ahmed  AL-BADANI, Shawki Ali Ahmed  90.0    SDGT   

                                             details   status  \
0                      DOB 04 Mar 1953; POB Nigeria.  FLAGGED   
1                                                NaN    CLEAR   
2  DOB 07 Nov 1985; POB Manizales, Caldas, Colomb...  FLAGGED   
3  DOB 01 Jan 1981; alt. DOB 01 Jan 1982; POB San...  FLAGGED   

             timestamp  
0

In [39]:
import pandas as pd
from datetime import datetime

STATE_PATH = "/kaggle/working/kyc_state.csv"
REPORT_PATH = "/kaggle/working/kyc_report.csv"


state = pd.read_csv(STATE_PATH, dtype=str).fillna("")

def make_explanation(row):
    if row["status"] == "FLAGGED":
        return (f"Name '{row['input_name']}' matched sanctioned entity '{row['matched_name']}' "
                f"with similarity {row['score']} under program '{row['program']}'.")
    else:
        return f"No relevant match found for '{row['input_name']}'."

state["explanation"] = state.apply(make_explanation, axis=1)


state.to_csv(REPORT_PATH, index=False)
print(f"Explainability report saved at {REPORT_PATH}")
state.head(10)


Explainability report saved at /kaggle/working/kyc_report.csv


,record_id,input_name,matched_name,score,program,details,status,timestamp,explanation
0,r1,John A. Doe,"OGUNGBUYI, Oluwole A.",85.5,SDNTK,DOB 04 Mar 1953; POB Nigeria.,FLAGGED,2025-10-23 12:16:51,Name 'John A. Doe' matched sanctioned entity '...
1,r2,Jane Q. Public,,,,,CLEAR,2025-10-23 12:16:53,No relevant match found for 'Jane Q. Public'.
2,r3,Jonathan Doe,"MUNOZ MEJIA, Jonathan",85.5,SDNTK,"DOB 07 Nov 1985; POB Manizales, Caldas, Colomb...",FLAGGED,2025-10-23 12:16:55,Name 'Jonathan Doe' matched sanctioned entity ...
3,r4,Ali Ahmed,"AL-BADANI, Shawki Ali Ahmed",90.0,SDGT,DOB 01 Jan 1981; alt. DOB 01 Jan 1982; POB San...,FLAGGED,2025-10-23 12:16:57,Name 'Ali Ahmed' matched sanctioned entity 'AL...


In [40]:
import time, pandas as pd, os
from rapidfuzz import process, fuzz
from datetime import datetime
from PIL import Image
import pytesseract, subprocess

SDN_PATH = "/kaggle/input/final-kyc/sdn_clean.csv"
INCOMING_PATH = "/kaggle/working/incoming_names.csv"
STATE_PATH = "/kaggle/working/kyc_state.csv"
REPORT_PATH = "/kaggle/working/kyc_report.csv"

def extract_name_from_doc(path):
    if not path or not os.path.exists(path): return ""
    if path.lower().endswith(".pdf"):
        tmp_dir = "/kaggle/working/tmp_pdf"
        os.makedirs(tmp_dir, exist_ok=True)
        subprocess.run(["pdftoppm","-r","200","-png",path,os.path.join(tmp_dir,"page")],check=True)
        return "\n".join(pytesseract.image_to_string(Image.open(os.path.join(tmp_dir,f)))
                         for f in sorted(os.listdir(tmp_dir)) if f.endswith(".png"))
    return pytesseract.image_to_string(Image.open(path))

def process_new_records():
    sdn = pd.read_csv(SDN_PATH, dtype=str).fillna("")
    inc = pd.read_csv(INCOMING_PATH, dtype=str).fillna("")
    state = pd.read_csv(STATE_PATH, dtype=str) if os.path.exists(STATE_PATH) else pd.DataFrame(columns=[
        "record_id","input_name","matched_name","score","program","details","status","timestamp"
    ])
    for _, row in inc.iterrows():
        if row["record_id"] in state["record_id"].values: continue
        name = row["name"] or extract_name_from_doc(row["doc_path"])
        match = process.extractOne(name, sdn["name"], scorer=fuzz.WRatio)
        ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if match and match[1] >= 85:
            hit = sdn.iloc[match[2]]
            status, score = "FLAGGED", f"{match[1]:.1f}"
            matched_name, prog, det = hit["name"], hit["program"], hit["details"]
        else:
            status, score, matched_name, prog, det = "CLEAR","","","",""
        state.loc[len(state)] = [row["record_id"], name, matched_name, score, prog, det, status, ts]
        print(f"{ts}  processed {row['record_id']} → {status}")
    state.to_csv(STATE_PATH, index=False)
    state["explanation"] = state.apply(
        lambda r: f"Name '{r['input_name']}' matched '{r['matched_name']}' (score {r['score']}) under {r['program']}"
        if r["status"]=="FLAGGED" else f"No match for '{r['input_name']}'", axis=1)
    state.to_csv(REPORT_PATH, index=False)


for _ in range(3):        
    process_new_records()
    print("refreshed; waiting 10 sec...\n")
    time.sleep(10)


refreshed; waiting 10 sec...

refreshed; waiting 10 sec...

refreshed; waiting 10 sec...

